In [ ]:
'''
k-correction demonstrator.
'''

In [ ]:
import sys
if './SelfCalGroupFinder/py/' not in sys.path:
    sys.path.append('./SelfCalGroupFinder/py/')

import numpy as np
import matplotlib.pyplot as plt
import SelfCalGroupFinder.py.kcorr.k_corrections as kc

In [ ]:
survey = 'Y1'
fpathN = '/pscratch/sd/a/amjsmith/Y1/LSS/iron/LSScats/v0.5/BGS_BRIGHT_N_clustering.dat.fits'
fpathS = '/pscratch/sd/a/amjsmith/Y1/LSS/iron/LSScats/v0.5/BGS_BRIGHT_S_clustering.dat.fits'
dat = kc.load_catalogues(fpathN, fpathS)

reg = 'N' # assume redshift limits same for N and S.
min_z_val = kc.selection(reg)['zmin']
max_z_val = kc.selection(reg)['zmax']

zmask = (dat['Z'] > min_z_val) & (dat['Z'] < max_z_val)
mask = zmask
dat = dat[mask]

In [ ]:
regions = ['N', 'S']

dat['K_R'] = -99.9

for reg in regions:
    regmask = (dat['reg'] == reg)
    kcorr_r  = kc.DESI_KCorrection(band='R', file='jmext', photsys=reg)
    
    dat['K_R'][regmask] = kcorr_r.k(dat['Z'][regmask], dat['REST_GMR_0P1'][regmask])

In [ ]:
dat

In [ ]:
def plot_kcorr(regions):
    # extract the default colour sequence to have more control of line colors
    prop_cycle = plt.rcParams['axes.prop_cycle']
    colors = prop_cycle.by_key()['color']

    for reg in regions:
        # set up the k-corrections for this photometry system region
        kcorr_r  = kc.DESI_KCorrection(band='R', file='jmext', photsys=reg)
        Sel=kc.selection(reg) # define selection parameters for this region
        z=np.linspace(0.0,0.6,300)
        icol=-1
        #for rest_GMR in np.array([0.39861393, 0.53434181, 0.6534462 , 0.76661587, 0.86391068,
       #0.93073082, 0.9832058 ]): #
            
        for rest_GMR in np.linspace(0.0,1.1,8):   
            GMR=rest_GMR*np.ones(z.size)
            icol += 1
            k=kcorr_r.k(z, GMR)
            label=reg+': G-R='+np.array2string(rest_GMR)
            plt.plot(z,k,label=label,color=colors[icol],linestyle=Sel['style']) 
    plt.xlabel('$z$')    
    plt.ylabel('$k^r(z)$')  
    plt.legend(loc=(1.04,0))
    
    plt.show()

In [ ]:
plot_kcorr(regions)